# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
D. Blain  ->  D. Blain  |  ['D. Blain']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
P. Boley  ->  P. Boley  |  ['P. Boley']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']


K. Lee  ->  K. Lee  |  ['K. Lee']
Arxiv has 66 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.18348


extracting tarball to tmp_2508.18348...

 done.
  0: tmp_2508.18348/aassymbols.tex, 579 lines
  1: tmp_2508.18348/ZTFJ1944+4557.tex, 542 lines
Retrieving document from  https://arxiv.org/e-print/2508.18358


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2508.18348/aassymbols.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2508.19053/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authorlist' from 'tmp_2508.19053/authorlist.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 131 bibliographic references in tmp_2508.19053/main.bbl.
Issues with the citations
syntax error in line 81: '=' expected
Retrieving document from  https://arxiv.org/e-print/2508.19170


extracting tarball to tmp_2508.19170...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.18366-b31b1b.svg)](https://arxiv.org/abs/2508.18366) | **The mid-infrared spectrum of $β$ Pictoris b. First VLTI/MATISSE interferometric observations of an exoplanet**  |
|| M. Houllé, et al. -- incl., <mark>D. Blain</mark>, <mark>G. Chauvin</mark>, <mark>P. Mollière</mark>, <mark>M. Ravet</mark>, <mark>P. Boley</mark>, <mark>L. Kreidberg</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| *21 pages, 17 figures. Accepted for publication in A&A*|
|**Abstract**|            Few spectra of directly-imaged exoplanets have been obtained in the mid-infrared (> 3 $\mu$m). This region is particularly rich in molecular spectral signatures, whose measurements can help recover atmospheric parameters and provide a better understanding of giant planet formation and atmospheric dynamics. In the past years, exoplanet interferometry with the VLTI/GRAVITY instrument has provided medium-resolution spectra of a dozen substellar companions in the near infrared. The 100-meter interferometric baselines allow for the stellar and planetary signals to be efficiently disentangled at close angular separations (< 0.3''). We aim to extend this technique to the mid-infrared using MATISSE, the VLTI's mid-infrared spectro-interferometer. We take advantage of the fringe tracking and off-axis pointing capabilities recently brought by the GRA4MAT upgrade. Using this new mode, we observed the giant planet $\beta$ Pictoris b in L and M bands (2.75-5 $\mu$m) at a spectral resolution of 500. We developed a method to correct chromatic dispersion and non-common paths effects in the fringe phase and modelled the planet astrometry and stellar contamination. We obtained a high-signal-to-noise spectrum of $\beta$ Pictoris b, showing the planet continuum in L (for the first time) and M bands, which contains broad absorption features of H$_2$O and CO. In conjunction with a new GRAVITY spectrum, we modelled it with the ForMoSA nested sampling tool and the Exo-REM grid of atmospheric models, and found a solar carbon-to-oxygen ratio in the planet atmosphere. This study opens the way to the characterization of fainter and closer-in planets with MATISSE, which could complement the JWST at angular separations too close for it to obtain exoplanet spectra. Starting in 2025, the new adaptive optics system brought by the GRAVITY+ upgrade will further extend the detection limits of MATISSE.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19053-b31b1b.svg)](https://arxiv.org/abs/2508.19053) | **WIde Separation Planets In Time (WISPIT): A Gap-clearing Planet in a Multi-ringed Disk around the Young Solar-type Star WISPIT 2**  |
|| R. F. v. Capelleveen, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| **|
|**Abstract**|            In the past decades several thousand exoplanet systems have been discovered around evolved, main-sequence stars, revealing a wide diversity in their architectures. To understand how the planet formation process can lead to vastly different outcomes in system architecture we have to study the starting conditions of planet formation within the disks around young stars. In this study we are presenting high resolution direct imaging observations with VLT/SPHERE of the young ($\sim$5 Myr), nearby ($\sim$133 pc), solar-analog designated as WISPIT 2($=$ TYC~5709-354-1). These observations were taken as part of our survey program that explores the formation and orbital evolution of wide-separation gas giants. WISPIT 2 was observed in four independent epochs using polarized light and total intensity observations. They reveal for the first time an extended (380 au) disk in scattered light with a multi-ringed sub-structure. We directly detect a young proto-planet WISPIT 2b, embedded in a disk gap and show that it is co-moving with its host star. Multiple SPHERE epochs demonstrate that it shows orbital motion consistent with Keplerian motion in the observed disk gap. Our $H$ and $K_s$-band photometric data are consistent with thermal emission from a young planet. By comparison with planet evolutionary models, we find a mass of the planet of $4.9^{+0.9}_{-0.6}$ Jupiter masses. This mass is also consistent with the width of the observed disk gap, retrieved from hydrodynamic models. WISPIT 2b is the first unambiguous planet detection in a multi-ringed disk, making the WISPIT 2 system the ideal laboratory to study planet-disk interaction and subsequent evolution.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.18348-b31b1b.svg)](https://arxiv.org/abs/2508.18348) | **Transiting Planetary Debris near the Roche Limit of a White Dwarf on a 4.97$\,$hr Orbit -- and its Vanishing**  |
|| J. A. Guidry, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| *The first two authors listed are both lead authors. 23 pages, 9 figures, accepted for publication in The Astrophysical Journal. For a Zenodo library housing all the follow-up time-series photometry we collected and analyzed, see this https URL*|
|**Abstract**|            We present the discovery of deep, irregular, periodic transits towards the white dwarf ZTF$\,$J1944$+$4557 using follow-up time-series photometry and spectroscopy from Palomar, Keck, McDonald, Perkins, and Lowell observatories. We find a predominant period of 4.9704$\,$hr, consistent with an orbit near the Roche limit of the white dwarf, with individual dips over 30$\%$ deep and lasting between 15 and 40 minutes. Similar to the first known white dwarf with transiting debris, WD$\,$1145$+$017, the transit events are well-defined with prominent out-of-transit phases where the white dwarf appears unobscured. Spectroscopy concurrent with transit photometry reveals the average Ca$\,$K equivalent width remains constant in and out of transit. The broadening observed in several absorption features cannot be reproduced by synthetic photospheric models, suggesting the presence of circumstellar gas. Simultaneous $g+r$- and $g+i$-band light curves from the CHIMERA instrument reveal no color dependence to the transit depths, requiring transiting dust grains to have sizes $s \gtrsim0.2\,\mu$m. The transit morphologies appear to be constantly changing at a rate faster than the orbital period. Overall transit activity varies in the system, with transit features completely disappearing during the seven months between our 2023 and 2024 observing seasons and then reappearing in 2025$~$March, still repeating at 4.9704$\,$hr. Our observations of the complete cessation and resumption of transit activity provide a novel laboratory for constraining the evolution of disrupted debris and processes like disk exhaustion and replenishment timescales at white dwarfs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.18538-b31b1b.svg)](https://arxiv.org/abs/2508.18538) | **Impact of gravity on changing magnetic field orientations in a sample of massive protostellar clusters observed with ALMA**  |
|| Q. Zhang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| *Accepted for publication in the Astrophysical Journal*|
|**Abstract**|            The magnetic field is integral to our understanding of the formation and dynamical evolution of molecular clouds and star formation within. We present a polarimetric survey of 17 massive protostellar cluster forming clumps, covered in 34 pointings in the 230-GHz window using the Atacama Large Millimeter/submillimeter Array (ALMA). The two array configurations, C43-1 and C43-4, probe linearly polarized dust emission, hence the plane-of-the-sky orientation of magnetic fields, at resolutions of 1\arcsec\ and 0\arcsec.4 that correspond to approximately 0.01pc core and $10^3$ au envelope scales, respectively. The relative orientations (ROs) of the magnetic field probed at two spatial scales are analyzed for the entire protostellar cluster sample and for a subset of objects in NGC 6334. We found a bimodal distribution of ROs with peaks at 0° (parallel) and 90°(orthogonal) for the entire sample combined as well as for NGC 6334. We investigate the physical origin of this bimodal distribution through a projected Rayleigh statistic (PRS) analysis in relation to column densities and local gravity in NGC 6334. We found an excess of parallel magnetic fields at column densities $> 10^{23}$ \cmm. The underlying cause of the RO distribution of the magnetic field is gravitational collapse at higher gas densities, which drags and reorients the magnetic field as shown in the alignment between the magnetic field and the direction of gravitational forces. The distribution of ROs observed here is consistent with the evolution of relative orientations of an initially sub-Alvénic cloud that becomes magnetically super-critical and super-Alvénic as the cloud collapses to form stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19170-b31b1b.svg)](https://arxiv.org/abs/2508.19170) | **Stellar Mass Assembly History of Massive Quiescent Galaxies since $z\sim4$: Insights from Spatially Resolved SED Fitting with JWST Data**  |
|| N. S. Haryana, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| *21 pages, 13 figures, submitted to ApJ*|
|**Abstract**|            Massive quiescent galaxies at high redshift show significantly more compact morphology than their local counterparts. To examine their internal structure across a wide redshift range and investigate potential redshift dependence, we performed spatially resolved SED fitting using pixedfit software on massive $(\log(M_*/M_\odot)\sim11)$ quiescent galaxies at $0<z<4$ with public James Webb Space Telescope and Hubble Space Telescope imaging data from the Public Release Imaging for Extragalactic Research and the Cosmic Evolution Early Release Science Survey. We find that at $z \sim 3.5$, the half-mass radius is about 5.4 times smaller than at $z \sim 0.5$. This growth is driven by stellar mass buildup in the outskirts ($r > 4$ kpc), while the central regions ($r \sim 1$ kpc) remain largely unchanged, with stellar mass surface density similar to local quiescent galaxies. The estimated star formation rates are too low to explain the stellar mass growth, indicating an additional stellar mass accumulation process, such as mergers, is necessary. We parameterize the size-mass relation of the most massive galaxies in our sample as $\log(R_{e,mass}) \propto \alpha \log(M_*)$, and find $\alpha = 2.67^{+1.14}_{-1.17}$ for $z\lessapprox2$, consistent with growth dominated by minor mergers, and $\alpha = 0.91^{+0.20}_{-0.16}$ for $z\gtrapprox2$, consistent with growth dominated by major mergers. These results indicate that massive quiescent galaxies originate from compact quenched systems and grow through combinations of minor and major mergers.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.18358-b31b1b.svg)](https://arxiv.org/abs/2508.18358) | **The Missing Hard Photons of Little Red Dots: Their Incident Ionizing Spectra Resemble Massive Stars**  |
|| B. Wang, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| *Submitted to ApJ; 25 pages, 14 figures, 1 table*|
|**Abstract**|            The nature of Little Red Dots (LRDs) has largely been investigated through their continuum emission, with lines assumed to arise from a broad-line region. In this paper, we instead use recombination lines to infer the intrinsic properties of the central engine of LRDs. Our analysis first reveals a tension between the ionizing properties implied from H$\alpha$ and HeII$\,\lambda$4686. The high H$\alpha$ EWs require copious H-ionizing photons, more than the bluest AGN ionizing spectra can provide. In contrast, HeII emission is marginally detected, and its low EW is, at most, consistent with the softest AGN spectra. The low HeII/H$\beta$ ($\sim10^{-2}$, $<20\times$ local AGN median) further points to an unusually soft ionizing spectrum. We extend our analysis to dense gas environments (the ``black-hole star'' hypothesis), and find that hydrogen recombination lines become optically thick and lose diagnostic power, but HeII remains optically thin and a robust tracer. Photoionization modeling with Cloudy rules out standard AGN accretion disk spectra. Alternative explanations include: exotic AGN with red rest-optical emission; a very high {\it average} optical depth ($>10$) from gas/dust; and/or soft ionizing spectra with abundant H-ionizing photons, consistent with e.g., a cold accretion disk or a composite of AGN and stars. The latter is an intriguing scenario since high hydrogen densities are highly conducive for star formation, and nuclear star clusters are found in the immediate vicinity of local massive black holes. While previous studies have mostly focused on features dominated by the absorbing hydrogen cloud, the HeII-based diagnostic proposed here represents a crucial step toward understanding the central engine of LRDs.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19046-b31b1b.svg)](https://arxiv.org/abs/2508.19046) | **Wide Separation Planets In Time (WISPIT): Discovery of a Gap H$α$ Protoplanet WISPIT 2b with MagAO-X**  |
|| L. M. Close, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-08-27*|
|*Comments*| *33 pages, 7 figures, Accepted for publication in ApJ Letters*|
|**Abstract**|            Excellent (<25 mas) H$_{\alpha}$ images of the star TYC 5709-354-1 led to the discovery of a rare H$_{\alpha}$ protoplanet. This star was discovered by the WISPIT survey to have a large multi-ring transitional disk, and is hereafter WISPIT 2. Our H$_{\alpha}$ images of 2025, April 13 and April 16 discovered an accreting (H$_{\alpha}$ in emission) protoplanet: WISPIT 2b (r=309.43$\pm$1.56 mas; (~54 au deprojected), PA=242.21$\pm$0.41 degrees) likely clearing a dust-free gap between the two brightest dust rings in the transitional disk. Our SNR=12.5 detection gave an H$_{\alpha}$ ASDI contrast of (6.5$\pm$0.5)x10$^{-4}$ and a H$_{\alpha}$ line flux of (1.29$\pm$0.28)x10$^{-15}$ erg/s/cm$^2$. We also present L' photometry from LBT/LMIRcam of the planet (L'=15.30$\pm$0.05 mag) which, when coupled with an age of 5.1$^{+2.4}_{-1.3}$ Myr, yields a planet mass estimate of 5.3$\pm$1.0 Mjup from the DUSTY evolutionary models. WISPIT 2b is accreting at 2.25$^{-0.17}_{+3.75}$x10$^{-12}$ Msun/yr. WISPIT 2b is very similar to the other H$_{\alpha}$ protoplanets in terms of mass, age, flux, and accretion rate. The inclination of the system (${\it i}$=44 degrees) is also, surprisingly, very similar to the other known H$\alpha$ protoplanet systems which all cluster from 37$\leq{\it i}\leq$52 degrees. We argue this clustering has only a ~1.0% (2.6 sigma) probability of occurring randomly, and so we speculate that magnetospherical accretion might have a preferred inclination range (~37-52 degrees) for the direct (cloud free, low extinction) line of sight to the H-alpha line formation/shock region. We also find at 110mas (~15au deprojected) a close companion candidate (CC1) which may be consistent with an inner dusty 9$\pm$4 Mjup planet.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.18366/./figures/full_spectrum_with_absorption.png', 'tmp_2508.18366/./figures/2022-11-09T053702_OB2_exp7_frame3_planet_bin_real_model_allframes_augmented.png', 'tmp_2508.18366/./figures/2022-11-09T053702_OB2_exp7_frame3_planet_bin_imag_model_allframes_augmented.png', 'tmp_2508.18366/./figures/refraction_model_comparison_star_jamescorr_v2.png', 'tmp_2508.18366/./figures/refraction_model_comparison_planet_jamescorr_v2.png']
copying  tmp_2508.18366/./figures/full_spectrum_with_absorption.png to _build/html/
copying  tmp_2508.18366/./figures/2022-11-09T053702_OB2_exp7_frame3_planet_bin_real_model_allframes_augmented.png to _build/html/
copying  tmp_2508.18366/./figures/2022-11-09T053702_OB2_exp7_frame3_planet_bin_imag_model_allframes_augmented.png to _build/html/
copying  tmp_2508.18366/./figures/refraction_model_comparison_star_jamescorr_v2.png to _build/html/
copying  tmp_2508.18366/./figures/refraction_model_comparison_planet_jamescorr_v2.png to _build/html/
exp

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The mid-infrared spectrum of $\beta$ Pictoris b: First VLTI/MATISSE interferometric observations of an exoplanet $\thanks{Based on public data released from the MATISSE commissioning observations at the VLT Interferometer under ESO Programme 60.A-9257(H), and NACO observations under ESO Programme 088.C-0196.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.18366-b31b1b.svg)](https://arxiv.org/abs/2508.18366)<mark>Appeared on: 2025-08-27</mark> -  _21 pages, 17 figures. Accepted for publication in A&A_

</div>
<div id="authors">

M. Houllé, et al. -- incl., <mark>D. Blain</mark>, <mark>G. Chauvin</mark>, <mark>P. Mollière</mark>, <mark>M. Ravet</mark>, <mark>P. Boley</mark>, <mark>L. Kreidberg</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:** Few spectra of directly-imaged exoplanets have been obtained in the mid-infrared ( $>3$ µm). This region is particularly rich in molecular spectral signatures, whose measurements can help recover atmospheric parameters and provide a better understanding of giant planet formation and atmospheric dynamics. In the past years, exoplanet interferometry with the VLTI/GRAVITY instrument has provided medium-resolution spectra of a dozen substellar companions in the near infrared. The 100-meter interferometric baselines allow for the stellar and planetary signals to be efficiently disentangled at close angular separations ( $<0.3$ "). We aim to extend this technique to the mid-infrared using MATISSE, the VLTI’s mid-infrared spectro-interferometer. We take advantage of the fringe tracking and off-axis pointing capabilities recently brought by the GRA4MAT upgrade. Using this new mode, we observed the giant planet $\beta$ Pictoris b in $L$ and $M$ bands (2.75--5 µm) at a spectral resolution of 500. We developed a method to correct chromatic dispersion and non-common paths effects in the fringe phase and modelled the planet astrometry and stellar contamination. We obtained a high-signal-to-noise spectrum of $\beta$ Pictoris b, showing the planet continuum in $L$ (for the first time) and $M$ bands, which contains broad absorption features of $H_2$ O and CO. In conjunction with a new GRAVITY spectrum, we modelled it with the \texttt{ForMoSA} nested sampling tool and the Exo-REM grid of atmospheric models, and found a solar carbon-to-oxygen ratio in the planet atmosphere. This study opens the way to the characterization of fainter and closer-in planets with MATISSE, which could complement the JWST at angular separations too close for it to obtain exoplanet spectra. Starting in 2025, the new adaptive optics system brought by the GRAVITY+ upgrade will further extend the detection limits of MATISSE.

</div>

<div id="div_fig1">

<img src="tmp_2508.18366/./figures/full_spectrum_with_absorption.png" alt="Fig12" width="100%"/>

**Figure 12. -** Spectrum of $\beta$ Pic b with MATISSE (black, this work) compared to other instruments: NACO (yellow, new data), GPI \citep[blue,][]{Chilcote2017}, GRAVITY (green, new data) and MIRI \citep[red,][]{Worthen2024}. Only the slope of the NACO spectrum can be compared to MATISSE as the absolute level was scaled manually. MATISSE data with a S/N < 5, located in strong telluric bands and not used in this work, are shown in grey. The absorption curves of $H_2$O, CO, and $CH_4$, calculated from the best-fitting Exo-REM model presented in Sect. \ref{sec:modeling}, are plotted at the top to visualize their impact on the spectrum. $H_2$O and CO leaves a visible pattern, but not $CH_4$, likely because it is dominated by $H_2$O absorption or buried in noise. (*fig:spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2508.18366/./figures/2022-11-09T053702_OB2_exp7_frame3_planet_bin_real_model_allframes_augmented.png" alt="Fig10.1" width="50%"/><img src="tmp_2508.18366/./figures/2022-11-09T053702_OB2_exp7_frame3_planet_bin_imag_model_allframes_augmented.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Real (left) and imaginary (right) parts of the measured planet-to-star coherent flux ratio $\mathcal{F}$ on the six UT baselines, during one 10-s frame. The error range is plotted in the background. The thick lines show the best-fit model as described in Sect. \ref{sec:astrometry}. It includes the star-planet modulation, the stellar speckle contamination (dashed blue curves), and the planet-to-star contrast assumption (dashed black curve). (*fig:model+dataampphase*)

</div>
<div id="div_fig3">

<img src="tmp_2508.18366/./figures/refraction_model_comparison_star_jamescorr_v2.png" alt="Fig2.1" width="50%"/><img src="tmp_2508.18366/./figures/refraction_model_comparison_planet_jamescorr_v2.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Measured differential phase (grey dots) in two MATISSE frames, one on the star (top) and the other on the planet (bottom). Overlaid are the models presented in this work using the air refractive indices of [ and Voronin (2017)]()(blue), [ and Mathar (2007)]()(green) and [ and Ciddor (1996)]()(red). In the absence of chromatic or non-common path OPD, the differential phase should be zero on the star or an oscillation on the planet.
     (*fig:phase-correction*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.18366"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\wis}{WISPIT\xspace}$
$\newcommand{\wisa}{WISPIT~2\xspace}$
$\newcommand{\wisb}{WISPIT~2b\xspace}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\mj}{\ensuremath{\mathrm{M_{Jup}}}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand{\mb}[1]{\textcolor{blue}{[#1]}}$
$\newcommand{\sfa}[1]{\textcolor{purple}{[#1]}}$
$\newcommand{\pmra}{\mu_{\alpha *}}$
$\newcommand{\pmdec}{\mu_{\delta}}$
$\newcommand{\masyr}{mas yr^{-1}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\cms}{cm s^{-2}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# WIde Separation Planets In Time (WISPIT): \ A gap-clearing planet in a multi-ringed disk around the young solar-type star WISPIT 2

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.19053-b31b1b.svg)](https://arxiv.org/abs/2508.19053)<mark>Appeared on: 2025-08-27</mark> - 

</div>
<div id="authors">

R. F. v. Capelleveen, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** In the past decades several thousand exoplanet systems have been discovered around evolved, main-sequence stars, revealing a wide diversity in their architectures.To understand how the planet formation process can lead to vastly different outcomes in system architecture we have to study the starting conditions of planet formation within the disks around young stars.In this study we are presenting high resolution direct imaging observations with VLT/SPHERE of the young ( $\sim$ 5 Myr), nearby ( $\sim$ 133 pc), solar-analog designated as $\wisa$ ( $=$ TYC 5709-354-1).These observations were taken as part of our survey program that explores the formation and orbital evolution of wide-separation gas giants.WISPIT 2 was observed in four independent epochs using polarized light and total intensity observations.They reveal for the first time an extended (380 au) disk in scattered light with a multi-ringed sub-structure.We directly detect a young proto-planet $\wisb$ , embedded in a disk gap and show that it is co-moving with its host star.Multiple SPHERE epochs demonstrate that it shows orbital motion consistent with Keplerian motion in the observed disk gap.Our $H$ and $K_s$ -band photometric data are consistent with thermal emission from a young planet.By comparison with planet evolutionary models, we find a mass of the planet of $4.9^{+0.9}_{-0.6}$ $\mj$ .This mass is also consistent with the width of the observed disk gap, retrieved from hydrodynamic models. $\wisb$ is the first unambiguous planet detection in a multi-ringed disk, making the $\wisa$ system the ideal laboratory to study planet-disk interaction and subsequent evolution.

</div>

<div id="div_fig1">

<img src="tmp_2508.19053/./planet-multi-panel_freeze.png" alt="Fig10" width="100%"/>

**Figure 10. -** All detections of the embedded planet $\wis$b in the various observation epochs and filters.
    We indicate the embedded planet position with a white, dotted circle.
    The coronagraph position is indicated with a gray, hashed mask.
    For the 2025 $K_s$-band data we show that the embedded planet is recovered with  classical and \ac{pca}-based \ac{adi} as well as in iRDI. (*fig:planet-images*)

</div>
<div id="div_fig2">

<img src="tmp_2508.19053/./WISPIT2_main_figure.png" alt="Fig9" width="100%"/>

**Figure 9. -** SPHERE/IRDIS observations of the $\wis$a system.
The gray, hashed disk in the image center indicates the size of the coronagraphic mask.
The differential imaging method and observed waveband for each image are indicated in the top left corner.
Blue-hued images reduced with the \ac{adi} or \ac{rdi}(50 principal components) methods are showing total intensity, sensitive to disk scattered light and thermal emission from embedded planets.
The gray-hued images are $Q_\phi$ images (reduced with the \ac{pdi} method) showing linearly polarized scattered light, not sensitive to thermal emission.
 (*fig:obs-results*)

</div>
<div id="div_fig3">

<img src="tmp_2508.19053/./ring3_bottom_side.png" alt="Fig17" width="100%"/>

**Figure 17. -** Zoom-in on the innermost disk ring (ring 3).
      We show polarized light $Q_\phi$ images in the gray color scheme in the left and middle panel and the total intensity cADI $K_s$-band image in the blue color scheme in the right panel.
      The middle panel shows the $Q_\phi$ image after image deconvolution was applied.
      Left and middle panel are displayed on a log scale due to the large dynamic range of the inner disk region while the right panel is on a linear scale.
      The gray hashed circle in the image center marks the area covered by the coronagraphic mask.
      We indicate with dotted lines (left and middle panel) and with a solid encircling ellipse (right panel) the signature of the inner disk bottom side. (*app:fig:disk bottom*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.19053"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

127  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
